In [24]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix
import nltk
from nltk import word_tokenize
import re
from sklearn import feature_extraction, feature_selection, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [25]:
data = pd.read_excel(r"C:\Users\gog10\GfK_spectacle_lenses_data.xlsx")

In [26]:
#Replace null values in the data
data.loc[data['main_text'].isnull(), 'main_text'] = 'Not Known'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3549 entries, 0 to 3548
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   item_id         3549 non-null   int64 
 1   main_text       3549 non-null   object
 2   country_name    3549 non-null   object
 3   retailer_pg     3549 non-null   object
 4   mdm_brand_text  3549 non-null   object
 5   mdm_model_text  3549 non-null   object
dtypes: int64(1), object(5)
memory usage: 166.5+ KB


In [27]:
#Get counts for brand text
data['mdm_brand_text'].value_counts()

ESSILOR           1266
HOYA               651
ZEISS              421
INDO               292
RODENSTOCK         279
SYNCHRONY          164
RUPP & HUBRACH     127
MAILSHOP            81
PRATS               57
SHAMIR              42
MEDICAL VISION      30
NIKA                29
KODAK               19
SEIKO               15
NIKON               11
RAY BAN             10
EYEART              10
TOKAI                9
WETZLICH             9
AMERICAN OPTIC       7
L.O.A.               7
BBGR                 7
VISALL               6
Name: mdm_brand_text, dtype: int64

In [28]:
#Delete columns not needed for model
data.drop(['item_id','country_name', 'retailer_pg', 'mdm_model_text'], axis = 'columns', inplace=True)

In [29]:
data.head()

,main_text,mdm_brand_text
0,ID1.60LIFESTL3IURBAN(RX),HOYA
1,ID1.60LIFESTL3IURBAN(RX),HOYA
2,ID1.60LIFESTL3IURBAN(RX),HOYA
3,ID1.60LIFESTL3IURBAN(RX),HOYA
4,ID1.60LIFESTL3IURBAN(RX),HOYA


In [30]:
# rename columns for model
data = data.rename(columns={"mdm_brand_text":"y", "main_text":"text"})
data

,text,y
0,ID1.60LIFESTL3IURBAN(RX),HOYA
1,ID1.60LIFESTL3IURBAN(RX),HOYA
2,ID1.60LIFESTL3IURBAN(RX),HOYA
3,ID1.60LIFESTL3IURBAN(RX),HOYA
4,ID1.60LIFESTL3IURBAN(RX),HOYA
...,...,...
3544,CRISTALES 34212 MI MF XPERIENCE 1.5 BASIC HV P...,HOYA
3545,CRISTALES 34186 MI HILUX 1.6 SHV FAB MONOFOCAL...,HOYA
3546,CRISTALES 34317 NULUX ACTIVE TF 1.5 HV ORGÁNIC...,HOYA
3547,CRISTALES 34244 MI LIFESTYLE 3I 1.5 HVLL BC PR...,HOYA


In [31]:
#list of stop words from nltk
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [32]:
#create a function to remove these stop words, remove certain characters, stem the text and lemmatize the text
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [33]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [34]:
#Apply the function 
data["text_clean"] = data["text"].apply(lambda x: 
          preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
data.head()

,text,y,text_clean
0,ID1.60LIFESTL3IURBAN(RX),HOYA,ID160LIFESTL3IURBANRX
1,ID1.60LIFESTL3IURBAN(RX),HOYA,ID160LIFESTL3IURBANRX
2,ID1.60LIFESTL3IURBAN(RX),HOYA,ID160LIFESTL3IURBANRX
3,ID1.60LIFESTL3IURBAN(RX),HOYA,ID160LIFESTL3IURBANRX
4,ID1.60LIFESTL3IURBAN(RX),HOYA,ID160LIFESTL3IURBANRX


In [35]:
## split dataset
data_train, data_test = model_selection.train_test_split(data, test_size=0.3)
## get target
y_train = data_train["y"].values
y_test = data_test["y"].values

In [36]:
## Count (classic BoW)
vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,2))

## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

In [37]:
texts = data_train["text_clean"]
vectorizer.fit(texts)
X_train = vectorizer.transform(texts)
dic_vocabulary = vectorizer.vocabulary_

In [38]:
vectorizer.get_feature_names()

['000',
 '000 0000',
 '000 5º',
 '000 ess',
 '000 loa',
 '0000',
 '0000 0000',
 '0000 0025',
 '0000 0050',
 '0000 0075',
 '0000 0100',
 '0000 0125',
 '0000 0150',
 '0000 0175',
 '0000 0200',
 '0000 0225',
 '0000 0250',
 '0000 0275',
 '0000 0300',
 '0000 0325',
 '0000 0350',
 '0000 0375',
 '0000 0400',
 '0000 0750',
 '0000 200',
 '0000 2250',
 '0000 300',
 '0000 airwear',
 '000202',
 '000202 masterlux',
 '001101',
 '001101 hifin',
 '001102',
 '001102 hifin',
 '0016',
 '0025',
 '0025 0050',
 '0025 0100',
 '0025 0125',
 '0025 0250',
 '0025 0275',
 '0025 0300',
 '0025 125',
 '0025 250',
 '0025 275',
 '0025 ormix',
 '004',
 '004 0125',
 '005',
 '005 0025',
 '005 0075',
 '005 0225',
 '0050',
 '0050 0000',
 '0050 0025',
 '0050 0050',
 '0050 0075',
 '0050 0100',
 '0050 0150',
 '0050 0175',
 '0050 0200',
 '0050 0225',
 '0050 0250',
 '0050 0300',
 '0050 0375',
 '0050 0400',
 '0050 0550',
 '0050 0750',
 '0050 125',
 '0050 200',
 '0050 250',
 '0050 airwear',
 '0050 aiwear',
 '0050 crizal',
 '0050 

In [39]:
vectorizer.vocabulary_

{'ese': 3474,
 'g8j634': 3808,
 '675': 2053,
 'ese g8j634': 3492,
 'g8j634 ese': 3809,
 'ese 675': 3483,
 '318100': 1462,
 'ese 318100': 3478,
 '318100 ese': 1465,
 'essilor': 3575,
 'lineis': 4573,
 '174': 1064,
 'transitions': 6137,
 '75': 2205,
 'crizal': 3019,
 'forte': 3774,
 'uv': 6229,
 'essilor lineis': 3586,
 'lineis 174': 4574,
 '174 transitions': 1081,
 'transitions 75': 6139,
 '75 crizal': 2209,
 'crizal forte': 3024,
 'forte uv': 3777,
 'balansis': 2573,
 '160': 874,
 'rx': 5671,
 'balansis 160': 2574,
 '160 rx': 911,
 '3es': 1562,
 '269100': 1308,
 '259': 1278,
 '3es 269100': 1567,
 '269100 3es': 1310,
 '3es 259': 1566,
 '485100': 1702,
 'varilux': 6270,
 'comfort': 2792,
 '30': 1415,
 'eye': 3703,
 'protect': 5469,
 'system': 6068,
 'orma': 5098,
 '15': 615,
 'ojo': 5014,
 'der': 3209,
 'esf': 3498,
 'cil': 2726,
 'eje': 3340,
 '90': 2361,
 'adic': 2450,
 '25': 1241,
 'diam': 3239,
 '6570': 2006,
 '485100 varilux': 1703,
 'varilux comfort': 6274,
 'comfort 30': 2796,
 '3

In [40]:
array_of_feature = X_train.toarray()
array_of_feature

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
for cat in np.unique(y):
   print("# {}:".format(cat))
   print("  . selected features:",
         len(data_features[data_features["y"]==cat]))
   print("  . top features:", ",".join(
data_features[data_features["y"]==cat]["feature"].values[:10]))
   print(" ")

# AMERICAN OPTIC:
  . selected features: 29
  . top features: 16 hcm,16 photofg,16 photomarron,7075 bas,american,american optical,bas,hcm,hcm zeiss,hmc 6570
 
# BBGR:
  . selected features: 22
  . top features: 17,17 60,17 65,17 diams,17 stock,60 diams,65 diams,diams,diams stock,especificarunem
 
# ESSILOR:
  . selected features: 295
  . top features: crizal,ormix,ormix 16,prevencia,ess,crizal prevencia,varilux,16 crizal,eps,ese
 
# EYEART:
  . selected features: 41
  . top features: 16 ultra,160monofocalorganico0hmcsin,160monofocalorganico0hmcsin color,161 hmc,174 shmc,174 super,174monofocalorganico0shmcsin,174monofocalorganico0shmcsin color,2540,2540 orlite
 
# HOYA:
  . selected features: 286
  . top features: eyas,hilux,id160lifestl3iurbanrx,lifestyle,rx,160,id,balansis,balansis 160,160 rx
 
# INDO:
  . selected features: 497
  . top features: basic,durcap,indosol,indosol basic,superfin,activa_rvd,natural10,indofin,basic gris,indofin 16
 
# KODAK:
  . selected features: 74
  . top 

In [42]:
classifier = naive_bayes.MultinomialNB()

## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train)
## test
X_test = data_test["text_clean"].values
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

In [43]:
classes = np.unique(y_test)
y_test_array = pd.get_dummies(y_test, drop_first=False).values

precision = precision_score(y_test, predicted, average='micro', zero_division=0)
print('Precision score: {0:0.2f}'.format(precision))

recall = recall_score(y_test, predicted, average='micro', zero_division=0)
print('Recall score: {0:0.2f}'.format(recall))

f1 = f1_score(y_test, predicted, average='micro', zero_division=0)
print('f1 score: {0:0.2f}'.format(recall))

Precision score: 0.88
Recall score: 0.88
f1 score: 0.88
